# <font color='blue'> Trabalho I Computação gráfica

## Alunos
### Felipe Destaole / 13686768
### João Pedro Farjoun Silva / 13731319

In [1]:
# !pip install glfw
# !pip install pyopengl


### Primeiro, vamos importar as bibliotecas necessárias.
Verifique no código anterior um script para instalar as dependências necessárias (OpenGL e GLFW) antes de prosseguir.

In [2]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import math

### Inicializando janela

In [3]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(1920, 1080, "Fundo do mar", None, None)
glfw.make_context_current(window)

### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Nós estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Iremos aprender GLSL conforme a necessidade do curso. Usarmos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec2.
* Definindo uma variável chamada mat_transformation do tipo mat4 (matriz 4x4).
* Usamos vec2, pois nosso programa (na CPU) irá enviar apenas duas coordenadas para plotar um ponto. Podemos mandar três coordenadas (vec3) e até mesmo quatro coordenadas (vec4).
* void main() é o ponto de entrada do nosso programa (função principal)
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso nós combinamos nossa variável vec2 com uma variável vec4. Além disso, nós modificamos nosso vetor com base em uma matriz de transformação, conforme estudado na Aula05.

In [4]:
vertex_code = """
        attribute vec3 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,1.0);
        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal)
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

### Possibilitando modificar a cor.

Nos exemplos anteriores, a variável gl_FragColor estava definida de forma fixa (com cor R=0, G=0, B=0).

Agora, nós vamos criar uma variável do tipo "uniform", de quatro posições (vec4), para receber o dado de cor do nosso programa rodando em CPU.

In [5]:
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [6]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [7]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [8]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [9]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [10]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [11]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, nós compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


### Vertices Baiacu(Esfera com triangulos)

In [12]:
PI = 3.141592
r = 0.3 # raio
num_sectors = 32 # qtd de sectors (longitude)
num_stacks = 32 # qtd de stacks (latitude)


# grid sectos vs stacks (longitude vs latitude)
sector_step=(PI*2)/num_sectors # variar de 0 até 2π
stack_step=(PI)/num_stacks # variar de 0 até π

# Entrada: angulo de longitude, latitude, raio
# Saida: coordenadas na esfera
def F(u,v,r):
    x = r*math.sin(v)*math.cos(u)
    y = r*math.sin(v)*math.sin(u)
    z = r*math.cos(v) - 0
    return (x,y,z)

# vamos gerar um conjunto de vertices representantes poligonos
# para a superficie da esfera.
# cada poligono eh representado por dois triangulos
vertices_baiacu = []
for i in range(0,num_sectors): # para cada sector (longitude)
    for j in range(0,num_stacks): # para cada stack (latitude)
        
        
        
        u = i * sector_step # angulo setor
        v = j * stack_step # angulo stack
        
        un = 0 # angulo do proximo sector
        if i+1==num_sectors:
            un = PI*2
        else: un = (i+1)*sector_step
            
        vn = 0 # angulo do proximo stack
        if j+1==num_stacks:
            vn = PI
        else: vn = (j+1)*stack_step
        
        # vertices do poligono
        p0=F(u, v, r)
        p1=F(u, vn, r)
        p2=F(un, v, r)
        p3=F(un, vn, r)
        
        # triangulo 1 (primeira parte do poligono)
        vertices_baiacu.append(p0)
        vertices_baiacu.append(p2)
        vertices_baiacu.append(p1)
        
        # triangulo 2 (segunda e ultima parte do poligono)
        vertices_baiacu.append(p3)
        vertices_baiacu.append(p1)
        vertices_baiacu.append(p2)

vertices_baiacu += [
    [+0.05,  0.0, 0.0],  # Vértice 1 (ponta da cauda)
    [-0.4,  0.3, 0.0],  # Vértice 2 (parte superior da cauda)
    [-0.4, -0.3, 0.0],  # Vértice 3 (parte inferior da cauda)
]

### Vertices estrela

In [13]:
vertices_estrela = [
 #parte de tras
 [-0.90, 0.25,0],
 [-0.31, 0.34,0],
 [0.0, 0.0,0.5],
 [0.0, 0.0,0.5],
 [-0.31, 0.34,0],
 [0.0, 0.94,0],
 [0.0, 0.94,0],
 [0.21, 0.34,0],
 [0.0, 0.0,0.5],
 [0.0, 0.0,0.5],
 [0.21, 0.34,0],
 [0.90, 0.25,0],
 [0.0, 0.0,0.5],
 [0.5, -0.25,0],
 [0.90, 0.25,0],
 [0.0, 0.0,0.5],
 [0.55, -0.85,0],
 [0.5, -0.25,0],
 [0.0, 0.0,0.5],
 [0.0, -0.56,0],
 [0.55, -0.85,0],
 [0.0, 0.0,0.5],
 [-0.55, -0.85,0],
 [0.0, -0.56,0],
 [0.0, 0.0,0.5],
 [-0.5, -0.25,0],
 [-0.55, -0.85,0],
 [0.0, 0.0,0.5],
 [-0.5, -0.25,0],
 [-0.90, 0.25,0],

 #parte frontal
 [-0.90, 0.25,0],
 [-0.31, 0.34,0],
 [0.0, 0.0,-0.5],
 [0.0, 0.0,-0.5],
 [-0.31, 0.34,0],
 [0.0, 0.94,0],
 [0.0, 0.94,0],
 [0.21, 0.34,0],
 [0.0, 0.0,-0.5],
 [0.0, 0.0,-0.5],
 [0.21, 0.34,0],
 [0.90, 0.25,0],
 [0.0, 0.0,-0.5],
 [0.5, -0.25,0],
 [0.90, 0.25,0],
 [0.0, 0.0,-0.5],
 [0.55, -0.85,0],
 [0.5, -0.25,0],
 [0.0, 0.0,-0.5],
 [0.0, -0.56,0],
 [0.55, -0.85,0],
 [0.0, 0.0,-0.5],
 [-0.55, -0.85,0],
 [0.0, -0.56,0],
 [0.0, 0.0,-0.5],
 [-0.5, -0.25,0],
 [-0.55, -0.85,0],
 [0.0, 0.0,-0.5],
 [-0.5, -0.25,0],
 [-0.90, 0.25,0]
]

### Vertices do corpo do peixe(elipse) e a cauda(triangulo)

In [14]:
# Número de vértices para o corpo do peixe
num_vertices_circulo = 33
raio = 0.4

vertices_corpo = []

for i in range(num_vertices_circulo):
    angulo = 2 * math.pi * i / num_vertices_circulo
    x = math.cos(angulo) * raio
    y = math.sin(angulo) * raio
    vertices_corpo.append([x, y,0.3])

vertices_cauda = [
    [+0.05,  0.0, 0.0],  # Vértice 1 (ponta da cauda)
    [-0.4,  0.3, 0.0],  # Vértice 2 (parte superior da cauda)
    [-0.4, -0.3, 0.0],  # Vértice 3 (parte inferior da cauda)
]



### Vertices do ceu(retangulo)

In [15]:
vertice_ceu = []

vertice_ceu = [
    [+2.0, -0.7, 0.2],
    [+2.0, +0.5, 0.2],
    [-2.0, -0.7, 0.2],
    [-2.0, +0.5, 0.2],
]

### Vertices do sol(esfera com triangulos)

In [16]:
num_vertices = 50  # pontos para formar o círculo
num_spikes = num_vertices  # número de espinhos
raio = 0.1
comprimento_espinhos = 0.2
angulo = 0.0

# Listas de vértices
vertices_sol = []
vertices_espinhos = []

# Gera os vértices do círculo
for i in range(num_vertices):
    angulo += 2 * math.pi / num_vertices
    x = math.cos(angulo) * raio
    y = math.sin(angulo) * raio
    vertices_sol.append([x, y, 0.0])

    # Adiciona vértices dos espinhos
    x_espinho = math.cos(angulo) * (raio + comprimento_espinhos)
    y_espinho = math.sin(angulo) * (raio + comprimento_espinhos)
    vertices_espinhos.append([x, y, 0])
    vertices_espinhos.append([x_espinho, y_espinho, 0])




### Vertices de um barco

In [17]:
vertices_barco = []

vertices_barco = [
    
    [-0.96, 0.0, -0.1],[-0.67, -0.83, -0.1],
    [-0.01, -0.14, -0.1],[-0.01, -0.14, -0.1],
    [0.65, -0.83, -0.1],[-0.67, -0.83, -0.1],
    [-0.01, -0.14, -0.1],[0.94, 0.00, -0.1],
    [0.65, -0.83, -0.1],[-0.46, -0.09, -0.1],
    [-0.01, 0.71, -0.1],[-0.01, -0.14, -0.1],
    [-0.01, 0.71, -0.1],[0.41, -0.09, -0.1],
    [-0.01, -0.14, -0.1],
]


In [18]:
obj_vert = []


obj_vert.append(vertice_ceu)
obj_vert.append(vertices_espinhos)
obj_vert.append(vertices_corpo)
obj_vert.append(vertices_cauda)
obj_vert.append(vertices_estrela)
obj_vert.append(vertices_barco)
obj_vert.append(vertices_baiacu)

obj_vert = np.concatenate(obj_vert)

In [19]:
vertices = np.zeros(len(obj_vert), [("position", np.float32, 3)])

In [20]:
vertices['position'] = obj_vert

### Para enviar nossos dados da CPU para a GPU, precisamos requisitar um slot.

In [21]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)


### Abaixo, nós enviamos todo o conteúdo da variável vertices.

Veja os parâmetros da função glBufferData [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [22]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Associando variáveis do programa GLSL (Vertex Shaders) com nossos dados

Primeiro, definimos o byte inicial e o offset dos dados.

In [23]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)


Em seguida, soliciamos à GPU a localização da variável "position" (que guarda coordenadas dos nossos vértices). Nós definimos essa variável no Vertex Shader.

In [24]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

A partir da localização anterior, nós indicamos à GPU onde está o conteúdo (via posições stride/offset) para a variável position (aqui identificada na posição loc).

Outros parâmetros:

* Definimos que possui duas coordenadas
* Que cada coordenada é do tipo float (GL_FLOAT)
* Que não se deve normalizar a coordenada (False)

Mais detalhes: https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glVertexAttribPointer.xhtml

In [25]:
glVertexAttribPointer(loc, 3, GL_FLOAT, False, stride, offset)

###  Vamos pegar a localização da variável color (uniform) para que possamos alterá-la em nosso laço da janela!

In [26]:
loc_color = glGetUniformLocation(program, "color")
loc_transform = glGetUniformLocation(program, "mat_transformation")

### Capturando eventos de teclado e modificando variáveis para a matriz de transformação

In [27]:
def multiplica_matriz(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c

In [28]:
#### O código dessa célula não está sendo usado.

def key_event(window,key,scancode,action,mods):
    global vel_x, vel_y, angulo, tam, flag, polygon_mode
    max_velo = 0.1
    max_ang = 0.1
    
    if key == 265: vel_y += 0.01 #cima
    if key == 264: vel_y -= 0.01 #baixo
    if key == 263: vel_x -= 0.01 #esquerda
    if key == 262: vel_x += 0.01 #direita



    if key == 65 and angulo < max_ang: angulo += 0.001 #a
    if key == 83 and angulo > -max_ang: angulo -= 0.001 #s

    if key == 90: tam += 0.01 
    if key == 88: tam -= 0.01

    if key == 80 and action == glfw.PRESS:
        if polygon_mode == GL_FILL:
            polygon_mode = GL_LINE  # Modo wireframe
        else:
            polygon_mode = GL_FILL  # Modo preenchido
    
glfw.set_key_callback(window,key_event)


def mouse_event(window,button,action,mods):
    
    print('[mouse event] button=',button)
    print('[mouse event] action=',action)
    print('[mouse event] mods=',mods)
    print('-------')
    if button == 0:
        print('cliquei no botao mouse esquerdo...')

glfw.set_mouse_button_callback(window,mouse_event)


In [29]:
def escala(sx,sy,sz):
    return np.array([sx, 0.0, 0.0, 0.0,
                     0.0, sy, 0.0, 0.0,
                     0.0, 0.0, sz, 0.0,
                     0.0, 0.0, 0.0, 1.0], np.float32)

In [30]:
def translacao(tx,ty,tz):
    return np.array([1.0, 0.0, 0.0, tx,
                     0.0, 1.0, 0.0, ty,
                     0.0, 0.0, 1.0, tz,
                     0.0, 0.0, 0.0, 1.0], np.float32)

In [31]:
def rotacao(theta_x,theta_y,theta_z):
    mat_rotation_z = np.array([
        np.cos(theta_z), -np.sin(theta_z), 0.0, 0.0, 
        np.sin(theta_z), np.cos(theta_z),  0.0, 0.0, 
        0.0,           0.0,            1.0, 0.0, 
        0.0,           0.0,            0.0, 1.0
        ], dtype=np.float32)

    mat_rotation_x = np.array([
        1.0,  0.0,            0.0,           0.0,
        0.0,  np.cos(theta_x), -np.sin(theta_x),  0.0, 
        0.0,  np.sin(theta_x),  np.cos(theta_x),  0.0, 
        0.0,  0.0,            0.0,           1.0
        ], dtype=np.float32)

    mat_rotation_y = np.array([
        np.cos(theta_y),  0.0,  np.sin(theta_y), 0.0, 
        0.0,            1.0,  0.0,          0.0, 
        -np.sin(theta_y), 0.0,  np.cos(theta_y), 0.0, 
        0.0,            0.0,  0.0,          1.0
        ], dtype=np.float32)

    
    mat_transform = multiplica_matriz(mat_rotation_z,mat_rotation_y)
    mat_transform = multiplica_matriz(mat_rotation_x,mat_transform)

    return mat_transform

### Nesse momento, nós exibimos a janela!


In [32]:
glfw.show_window(window)

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.

A novidade agora é a função glDrawArrays()

Tal função recebe o tipo de primitiva (GL_TRIANGLES), o índice inicial do array de vértices (vamos mostrar todos os três vértices, por isso começamos com 0) e a quantidade de vértices ( len(vertices) ).

### Dividindo em funções os objetos que temos que gerar

In [33]:
def background():
    glClearColor(0.0, 0.2, 0.5, 1.0)
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT) 
    
    glUniform4f(loc_color, 1.0, 1.0, 1.0, 1.0) #ceu
    glUniformMatrix4fv(loc_transform, 1, GL_TRUE, translacao(1.0, 1.0, 0))
    glDrawArrays(GL_TRIANGLE_STRIP, 0, len(vertice_ceu))

    glUniform4f(loc_color, 0.94, 0.9, 0.55, 1.0) #areia
    glUniformMatrix4fv(loc_transform, 1, GL_TRUE, translacao(1.0, -1.4, 0.1))
    glDrawArrays(GL_TRIANGLE_STRIP, 0, len(vertice_ceu))


    glUniform4f(loc_color, 1.0, 0.95, 0.0, 1.0) #sol
    glUniformMatrix4fv(loc_transform, 1, GL_TRUE, translacao(0.9, 0.9, -0.1))
    glDrawArrays(GL_POLYGON, len(vertice_ceu), len(vertices_espinhos))

In [34]:
def peixe():
    global inicio
    peixe_x = 0.3
    peixe_y = -0.5


    glUniform4f(loc_color, 1.0, 0.5, 0.31, 1.0) #peixe-corpo
    glUniformMatrix4fv(loc_transform, 1, GL_TRUE, multiplica_matriz(translacao(peixe_x, peixe_y, 0),escala(0.3,0.3,0)))
    glDrawArrays(GL_POLYGON, inicio, len(vertices_corpo))

    glUniform4f(loc_color, 1.0, 1.0, 1.0, 1.0) #peixe-olho
    glUniformMatrix4fv(loc_transform, 1, GL_TRUE, multiplica_matriz(translacao(peixe_x + 0.07, peixe_y + 0.06, -0.01),escala(0.04,0.04,0)))
    glDrawArrays(GL_POLYGON, inicio, len(vertices_corpo))

    glUniform4f(loc_color, 0.0, 0.0, 0.0, 1.0) #peixe-olho-parte-preta
    glUniformMatrix4fv(loc_transform, 1, GL_TRUE, multiplica_matriz(translacao(peixe_x + 0.07, peixe_y + 0.06, -0.02),escala(0.03,0.03,0)))
    glDrawArrays(GL_POLYGON, inicio, len(vertices_corpo))

    inicio += len(vertices_corpo)

    glUniform4f(loc_color, 1.0, 0.5, 0.31, 1.0) #peixe-cauda
    glUniformMatrix4fv(loc_transform, 1, GL_TRUE, multiplica_matriz(translacao(peixe_x - 0.05, peixe_y, 0),escala(0.3,0.3,0)))
    glDrawArrays(GL_POLYGON, inicio, len(vertices_cauda))

In [35]:
def barco(mat_barco):
    glUniform4f(loc_color, 0.75, 0.75, 0.75, 1.0) #barco
    glUniformMatrix4fv(loc_transform, 1, GL_TRUE, mat_barco)
    glDrawArrays(GL_TRIANGLES, inicio, len(vertices_barco))

In [36]:
def baiacu(mat_baiacu,cauda,olho):
    global inicio, baiacu_x,baiacu_y,baiacu_z,escalar
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_baiacu)

    for triangle in range(inicio,inicio+len(vertices),3):

        if triangle < (inicio+len(vertices))/2:
            R = 0.37 + (triangle - inicio)/20000
            G = 0.62 + (triangle - inicio)/20000
            B = 0.62 + (triangle - inicio)/20000
        else:
            R = 0.96
            G = 0.87
            B = 0.70
       
        glUniform4f(loc_color, R, G, B, 1.0)
        
        glDrawArrays(GL_TRIANGLES, triangle, 3)

    glUniform4f(loc_color, 1.0, 1.0, 1.0, 1.0) #olho
    glUniformMatrix4fv(loc_transform, 1, GL_TRUE, multiplica_matriz(translacao(baiacu_x * (2/3), baiacu_y*0.2, baiacu_z*2),escala(escalar*0.08,escalar*0.08,escalar*0.08)))
    glDrawArrays(GL_POLYGON, olho, len(vertices_corpo))

    glUniform4f(loc_color, 0.0, 0.0, 0.0, 1.0) #olho-parte-preta
    glUniformMatrix4fv(loc_transform, 1, GL_TRUE, multiplica_matriz(translacao(baiacu_x * (2/3), baiacu_y*0.2, baiacu_z*2.03),escala(escalar*0.06,escalar*0.06,escalar*0.06)))
    glDrawArrays(GL_POLYGON, olho, len(vertices_corpo))

    glUniform4f(loc_color, 1.0, 1.0, 1.0, 1.0) #olho
    glUniformMatrix4fv(loc_transform, 1, GL_TRUE, multiplica_matriz(translacao(baiacu_x * (2/3), baiacu_y*0.2, -baiacu_z*2),escala(escalar*0.08,escalar*0.08,escalar*0.08)))
    glDrawArrays(GL_POLYGON, olho, len(vertices_corpo))

    glUniform4f(loc_color, 0.0, 0.0, 0.0, 1.0) #olho-parte-preta
    glUniformMatrix4fv(loc_transform, 1, GL_TRUE, multiplica_matriz(translacao(baiacu_x * (2/3), baiacu_y*0.2, -baiacu_z*2.03),escala(escalar*0.06,escalar*0.06,escalar*0.06)))
    glDrawArrays(GL_POLYGON, olho, len(vertices_corpo))

In [37]:
def estrela(mat_estrela):
    global inicio

    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_estrela)
    
    for triangle in range(inicio,inicio+len(vertices_estrela),3):
        if triangle % 2 == 0:
            R = 1.0
            G = 0.39
            B = 0.28
        else:
            R = 1.0
            G = 0.5
            B = 0.31
       
        glUniform4f(loc_color, R, G, B, 1.0)
        
        glDrawArrays(GL_TRIANGLES, triangle, 3)

### Loop que exibe a janela e atualiza os objetos

In [38]:
import math

angulo = 0
tam = 0.5
theta_x = 0
theta_y = 0
theta_z = 0
vel_x = 0.0
vel_y = 0
s_x = 0
s_y = 0
polygon_mode = GL_FILL

glEnable(GL_DEPTH_TEST)


while not glfw.window_should_close(window):

    glfw.poll_events() 

    #paisagem
    
    background()

    inicio = len(vertice_ceu) + len(vertices_espinhos)

    #peixe

    peixe()

    inicio += len(vertices_cauda)

    #estrela

    theta_y += angulo #rotacionar em relacao ao eixo x

    mat_estrela = multiplica_matriz(translacao(-0.5,-0.73,0), escala(0.2,0.2,0.2))
    mat_estrela = multiplica_matriz(mat_estrela,rotacao(0,theta_y,0))

    estrela(mat_estrela)

    inicio += len(vertices_estrela)

    #barco

    s_x = vel_x

    mat_barco = multiplica_matriz(translacao(s_x,0.53, -0.15),escala(0.3,0.3,0))
    
    barco(mat_barco)

    inicio += len(vertices_barco)

    #baiacu

    baiacu_x = -0.3
    baiacu_y = -0.1
    baiacu_z = -0.3

    escalar = tam

    mat_baiacu = multiplica_matriz(translacao(baiacu_x,baiacu_y,baiacu_z),escala(escalar,escalar,escalar))

    cauda = len(vertice_ceu) + len(vertices_espinhos) + len(vertices_corpo)

    olho = len(vertice_ceu) + len(vertices_espinhos)

    baiacu(mat_baiacu,cauda,olho)
    
    glPolygonMode(GL_FRONT_AND_BACK, polygon_mode)
    
    glfw.swap_buffers(window)

glfw.terminate()